In [1]:
import os
import elasticsearch as es
from dotenv import load_dotenv
import requests
import csv
import os

In [2]:
load_dotenv()

True

In [3]:
urls = ["https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz",
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
        "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"]

In [4]:
len(urls)

45

In [5]:
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
client = es.Elasticsearch("http://10.11.12.69:9200", http_auth=(username, password), timeout=60, max_retries=10, retry_on_timeout=True)

In [6]:
client.ping()

True

In [7]:
client.indices.get_alias()

/Users/constantinkuehne/Documents/Uni/Semester5/ADP/ADP-4-load-data-into-elasticsearch/load-data/venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: this request accesses system indices: [.security-7, .transform-internal-007, .fleet-policies-7, .apm-custom-link, .kibana_task_manager_7.15.1_001, .kibana_7.15.1_001, .tasks, .async-search, .apm-agent-configuration, .kibana_security_session_1], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)
/Users/constantinkuehne/Documents/Uni/Semester5/ADP/ADP-4-load-data-into-elasticsearch/load-data/venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: this request accesses aliases with names reserved for system indices: [.kibana_task_manager_7.15.1, .kibana_7.15.1, .kibana_task_manager, .security, .data-frame-internal-3, .kibana, .fleet-policies], but in a future major versi

{'kibana_sample_data_logs': {'aliases': {}},
 '.monitoring-es-7-2022.01.04': {'aliases': {}},
 '.monitoring-kibana-7-2022.01.02': {'aliases': {}},
 'constantin_amazondataproject_v4': {'aliases': {}},
 '.siem-signals-default-000002': {'aliases': {'.siem-signals-default': {'is_write_index': True}}},
 '%{[@metadata][dest_index]}': {'aliases': {}},
 'kibana_sample_data_flights': {'aliases': {}},
 '.siem-signals-default-000001': {'aliases': {'.siem-signals-default': {'is_write_index': False}}},
 '.security-7': {'aliases': {'.security': {}}},
 '.transform-internal-007': {'aliases': {'.data-frame-internal-3': {}}},
 '.monitoring-kibana-7-2022.01.01': {'aliases': {}},
 'metrics-endpoint.metadata_current_default': {'aliases': {}},
 '.monitoring-kibana-7-2021.12.30': {'aliases': {}},
 '.monitoring-logstash-7-2022.01.02': {'aliases': {}},
 '.monitoring-kibana-7-2022.01.04': {'aliases': {}},
 '.monitoring-logstash-7-2021.12.29': {'aliases': {}},
 '.fleet-policies-7': {'aliases': {'.fleet-policies'

In [8]:
index_name = "constantin_amazondataproject_v6"

In [9]:
client.indices.create(index=index_name)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'constantin_amazondataproject_v6'}

In [47]:
client.search(index=index_name, query={"match_all": {}})

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [48]:
import pandas as pd

In [49]:
import csv
import numpy as np

In [50]:
body = {
    "mappings": {
        "properties": {
            "marketplace": { "type": "keyword" },
            "customer_id": { "type": "keyword"},
            "review_id": { "type": "keyword" },
            "product_id": { "type": "keyword" },
            "product_parent": { "type": "keyword" },
            "product_title": { "type": "text" },
            "star_rating": { "type": "byte" },
            "helpful_votes": { "type": "integer" },
            "total_votes": { "type": "integer" },
            "vine": { "type": "boolean" },
            "verified_purchase": { "type": "boolean" },
            "review_headline": { "type": "text" },
            "review_body": { "type": "text" },
            "review_date": { "type": "date" }
        }
    }
}

In [51]:
convert_to_bool = lambda x: True if x == "Y" else False
converters = {"vine": convert_to_bool, "verified_purchase": convert_to_bool}

In [52]:
dtypes = {
    "marketplace": "category",
    "customer_id": np.int64,
    "review_id": "object",
    "product_id": "object",
    "product_parent": np.int64,
    "product_title": "object",
    "star_rating": np.int8,
    "helpful_votes": np.int16,
    "total_votes": np.int16,
    "review_headline": "object",
    "review_body": "object",
}

In [53]:
global num_data_point
num_data_point = 0

In [54]:
def data_iterator(reader):
        # need to remove when using csv.DictReader
        global num_data_point
        for chunk in reader:
            chunk.replace(np.nan, "", inplace=True)
            for _, row in chunk.iterrows():
                doc = {
                    "marketplace": row["marketplace"],
                    "customer_id": row["customer_id"],
                    "review_id": row["review_id"],
                    "product_id": row["product_id"],
                    "product_parent": row["product_parent"],
                    "product_title": row["product_title"],
                    "product_category": row["product_category"],
                    "star_rating": row["star_rating"],
                    "helpful_votes": row["helpful_votes"],
                    "total_votes": row["total_votes"],
                    "vine": row["vine"],
                    "verified_purchase": row["verified_purchase"],
                    "review_headline": row["review_headline"],
                    "review_body": row["review_body"],
                    "review_date": row["review_date"],
                }
                num_data_point += 1
                yield doc

In [55]:
client.delete_by_query(index=index_name, body={"query": {"match_all": {}}})

{'took': 0,
 'timed_out': False,
 'total': 0,
 'deleted': 0,
 'batches': 0,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [56]:
client.search({"query": {"match_all": {}}}, index=index_name)

/var/folders/ps/dgdwqdr11mq34k0cc13_s_940000gn/T/ipykernel_66428/3111317702.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.search({"query": {"match_all": {}}}, index=index_name)


{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [57]:
from elasticsearch.helpers import bulk

d = "/Users/constantinkuehne/Downloads/archive"
for f in os.listdir(d):
    full_path = os.path.join(d, f)
    print(full_path)
    with open(full_path, mode="r") as f:
        # TODO: replace this with a version of csv.DictReader
        reader = pd.read_csv(
            f,
            delimiter="\t",
            quoting=csv.QUOTE_NONE,
            encoding="utf-8",
            chunksize=7000,
            parse_dates=["review_date"],
            converters=converters,
            dtype=dtypes,
        )
        res = bulk(client=client, index=index_name, actions=data_iterator(reader))
    print(res)

/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv
(1688884, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Sports_v1_00.tsv
(4850360, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Watches_v1_00.tsv
(960872, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Outdoors_v1_00.tsv
(2302401, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Automotive_v1_00.tsv
(3514942, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Grocery_v1_00.tsv
(2402458, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Office_Products_v1_00.tsv
(2642434, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Music_v1_00.tsv
(4751577, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Camera_v1_00.tsv
(1801974, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_us_Furniture_v1_00.tsv
(792113, [])
/Users/constantinkuehne/Downloads/archive/amazon_reviews_

BulkIndexError: ('500 document(s) failed to index.', [{'index': {'_index': 'constantin_amazondataproject_v5', '_type': '_doc', '_id': 'ak3z_H0BANdU85SxOb8F', 'status': 429, 'error': {'type': 'cluster_block_exception', 'reason': 'index [constantin_amazondataproject_v5] blocked by: [TOO_MANY_REQUESTS/12/disk usage exceeded flood-stage watermark, index has read-only-allow-delete block];'}, 'data': {'marketplace': 'US', 'customer_id': 39699233, 'review_id': 'R38Y2QW7R6S5NO', 'product_id': 'B001H5X7I4', 'product_parent': 826146509, 'product_title': 'Tropic Thunder', 'product_category': 'Video DVD', 'star_rating': 1, 'helpful_votes': 5, 'total_votes': 18, 'vine': False, 'verified_purchase': False, 'review_headline': 'watch something else...', 'review_body': "I watched a documentary about comics this week and it made me realize there are people and shows that are genuinely funny.  This movie is not one of them.  It is just a lot of bathroom humor--nothing really funny.  If you liked Ben Still

In [58]:
global num_data_point
print(num_data_point)

98417406
